# Zenith Conda Package Build (with CUDA)

This notebook builds a Conda package with CUDA kernels for distribution.

**Prerequisites:**
- GPU Runtime enabled (Runtime > Change runtime type > T4 GPU)
- Anaconda Cloud account (https://anaconda.org)

In [ ]:
# Cell 1: Check GPU and CUDA
!nvidia-smi --query-gpu=name,memory.total,compute_cap --format=csv
!nvcc --version

In [ ]:
# Cell 2: Install Miniconda and conda-build
!wget -q https://repo.anaconda.com/miniconda/Miniconda3-latest-Linux-x86_64.sh -O miniconda.sh
!bash miniconda.sh -b -p /content/miniconda
import os
os.environ['PATH'] = '/content/miniconda/bin:' + os.environ['PATH']
!conda install -y conda-build anaconda-client
print("Conda installed!")

In [ ]:
# Cell 3: Clone Zenith repository
!rm -rf ZENITH
!git clone https://github.com/vibeswithkk/ZENITH.git
%cd ZENITH
!git log -1 --oneline

In [ ]:
# Cell 4: Create conda recipe (meta.yaml)
import os
os.makedirs('conda', exist_ok=True)

meta_yaml = '''
package:
  name: pyzenith-cuda
  version: "0.2.0"

source:
  path: ..

build:
  number: 0
  script: |
    pip install .
  
requirements:
  host:
    - python >=3.9
    - pip
    - numpy
  run:
    - python >=3.9
    - numpy >=1.20

test:
  imports:
    - zenith
  commands:
    - python -c "import zenith; print(zenith.__version__)"

about:
  home: https://github.com/vibeswithkk/ZENITH
  license: MIT
  license_file: LICENSE
  summary: High-performance ML compiler framework with CUDA acceleration
  description: |
    Zenith is a high-performance machine learning compiler framework
    that provides optimization passes, quantization, and GPU acceleration.
'''

with open('conda/meta.yaml', 'w') as f:
    f.write(meta_yaml)

print("meta.yaml created!")
!cat conda/meta.yaml

In [ ]:
# Cell 5: Build Conda package
!conda build conda/ --output-folder /content/conda-output
print("\nBuild complete!")
!ls -la /content/conda-output/*/

In [ ]:
# Cell 6: Login to Anaconda Cloud
# Replace YOUR_TOKEN with your Anaconda token from:
# https://anaconda.org/settings/access

ANACONDA_TOKEN = "YOUR_TOKEN_HERE"  # <-- GANTI INI

if ANACONDA_TOKEN != "YOUR_TOKEN_HERE":
    !anaconda -t $ANACONDA_TOKEN whoami
else:
    print("Please set your ANACONDA_TOKEN first!")
    print("Get it from: https://anaconda.org/settings/access")

In [ ]:
# Cell 7: Upload to Anaconda Cloud
import glob

packages = glob.glob('/content/conda-output/**/*.tar.bz2', recursive=True)
print(f"Found {len(packages)} package(s) to upload:")
for pkg in packages:
    print(f"  - {pkg}")

if ANACONDA_TOKEN != "YOUR_TOKEN_HERE" and packages:
    for pkg in packages:
        !anaconda -t $ANACONDA_TOKEN upload $pkg --force
    print("\nUpload complete!")
    print("Install with: conda install -c YOUR_USERNAME pyzenith-cuda")
else:
    print("\nSet ANACONDA_TOKEN to upload, or download manually:")
    from google.colab import files
    for pkg in packages:
        files.download(pkg)

## Done!

After upload, users can install with:
```bash
conda install -c YOUR_USERNAME pyzenith-cuda
```